In [79]:
from lbfgsb_scipy import LBFGSBScipy
import torch
import torch.nn as nn
import numpy as np
import math

In [115]:
# define kernel function
def gaussian_kernel(x, y, gamma= 1): # [d, 1] * [d, 1] -> [1, 1]
      distance_squared = torch.norm(x-y, dim=-1)**2
      return torch.exp(-distance_squared / (gamma**2))

In [162]:
class NotearsRKHS(nn.Module):
    """n: number of samples, d: num variables"""
    def __init__(self, n, d):
        super(NotearsRKHS, self).__init__() # inherit  nn.Module
        self.d = d
        self.n = n

        # initialize coefficients alpha
        self.fc1_pos = nn.Linear(n, d, bias=False)  # fc1_pos.weight = [d ,n], fc1_pos(x) = x @ fc1_pos.weight^T
        self.fc1_neg = nn.Linear(n, d, bias=False)

        # Set positiv boundary for coefficients alpha to apply L-BFGS-B  
        self.fc1_pos.weight.bounds = self._bounds() 
        self.fc1_neg.weight.bounds = self._bounds()
        nn.init.zeros_(self.fc1_pos.weight)
        nn.init.zeros_(self.fc1_neg.weight)
        self.I = torch.eye(self.d)

    def _bounds(self):
        """Set boundary for coefficients alpha"""
        bounds = []
        for _ in range(self.n):
            for _ in range(self.d):
                bound = (0, None)
                bounds.append(bound)
        return bounds
    '''
    def kernel_matrix(self, x: np.array): # [n, d] -> [n, n]
       #get the kernel matrix s.t. K_{il} = k(x^i, x^l)
       x = torch.tensor(x, dtype=torch.float32, requires_grad=True)
       x1 = x.unsqueeze(-1)
       x1 = x1.repeat(1, 1, 2).transpose(1, 2)
       x2 = x.unsqueeze(0)
       x2 = x.repeat(2, 1, 1)
       K = gaussian_kernel(x1, x2)
       return K
    '''
    def gaussian_kernel_matrix_and_grad(self, x, gamma=1):
      x = torch.tensor(x, dtype=torch.float32, requires_grad=True)
    # Compute pairwise squared Euclidean distances using broadcasting
      diff = x.unsqueeze(1) - x.unsqueeze(0)
      sq_dist = torch.sum(diff ** 2, dim=2)

      # Compute the Gaussian kernel matrix
      K = torch.exp(-sq_dist / (gamma ** 2))

      # Compute the gradient of K with respect to X
      grad_K = -2 * diff / (gamma ** 2) * K.unsqueeze(2)

      return K, grad_K

    
    def forward(self, x: np.array): #[n,d] -> [n, d], forward(x)_{l,j} = estimation of x_j at lth observation 
      """
      x: data matrix of shape [n, d] (np.array)
      forward(x)_{l,j} = estimation of x_j at lth observation
      """
      K = self.gaussian_kernel_matrix_and_grad(x)[0]
      output = self.fc1_pos(K) - self.fc1_neg(K)
      return output
    
    def L_risk(self, x: np.array, penalty): # [1, 1]
      """compute the regularized iempirical L_risk of squared loss function, penalty: penalty for H_norm"""
      x_est = self.forward(x)
      K = self.gaussian_kernel_matrix_and_grad(x)[0] # [n, n]
      x = torch.tensor(x, dtype=torch.float32, requires_grad=True)
      squared_loss = 0.5 / self.n * torch.sum((x_est - x) ** 2)
      fc1_weight = self.fc1_pos.weight - self.fc1_neg.weight
      temp = torch.matmul(torch.matmul(fc1_weight, K), fc1_weight.t())
      diagonal = torch.sum(torch.diag(temp))
      regularized = penalty*diagonal 
      loss = squared_loss + regularized
      return loss
    """
    def grad_kernel(self, x) -> torch.Tensor: # [n, n, d] grad_storage[i, l, k]: gradient of k(x^i, x^l) wrt x^l_{k}
      #Get W from fc1 weights, take L2 norm of the gradient
      # get [n, n, d] gradient matrix of the kernel
      # Initialize a tensor to store gradients
      x = torch.tensor(x, dtype=torch.float32, requires_grad=True)
      grad_K = torch.zeros(self.n, self.n, self.d)
      # Compute gradients
      for i in range(self.n):
         for l in range(self.n):
            x_i = x[i,:]
            x_l = x[l,:]
            x_l.retain_grad()
            K_il = gaussian_kernel(x_i, x_l, gamma=1)
            # Zero out gradients in x, otherwise it accumulate the gradients
            if x_l.grad is not None:
                  x_l.grad.zero_()
            # Select individual scalar element from y
            # Perform backward pass on the scalar element
            K_il.backward()
            # Store the computed gradients
            grad_K[i, l] = x_l.grad.clone() 
      return grad_K
      """

    def fc1_to_adj(self, x: np.array) -> torch.Tensor: # [d, d]
       grad_K = self.gaussian_kernel_matrix_and_grad(x)[1]
       grad_K = grad_K.transpose(0, 2).unsqueeze(-1) # [d ,n, 1]
       fc1_weight = self.fc1_pos.weight - self.fc1_neg.weight # [d, n]
       weight = torch.matmul(fc1_weight, grad_K).squeeze(-1) # [d, n, d]
       weight = torch.sum(weight ** 2, dim = 1)/self.n # [d, d]
       return weight
    
    def h_func(self, x: np.array, t: float = 1.0) -> torch.Tensor: #[1, 1]
      """
      Parameters
      ----------
      t : float, optional
          Controls the domain of M-matrices, by default 1.0

      Returns
      -------
      torch.Tensor
          A scalar value of the log-det acyclicity function :math:`h(\Theta)`.
      """
      weight = self.fc1_to_adj(x)
      sign, logabsdet = torch.linalg.slogdet(t*self.I - weight)
      h = -sign * logabsdet + self.d * np.log(t)
      return h

In [193]:
def dual_ascent_step(model, x, lambda1, mu, rho, h, rho_max):
    """Perform one step of dual ascent in augmented Lagrangian."""
    h_new = None
    optimizer = LBFGSBScipy(model.parameters()) # since model inherit from nn.Module, model.parameters returns an iterator over all the parameters (weights, biases) of the model
    #X_torch = torch.from_numpy(X)
    print(rho)
    print(rho_max)
    while rho < rho_max:
        def closure():
            optimizer.zero_grad()
            #x_hat = model(x)
            loss = model.L_risk(x, penalty = lambda1)
            h_val = model.h_func(x)
            penalty = 0.5 * rho * h_val * h_val + mu * h_val
            #l2_reg = 0.5 * lambda2 * model.l2_reg()
            #l1_reg = lambda1 * model.fc1_l1_reg()
            primal_obj = loss + penalty 
            primal_obj.backward()
            return primal_obj
        optimizer.step(closure)  # NOTE: one step update model in-place
        #with torch.no_grad():
        h_new = model.h_func(x).item()
        if h_new > 0.25 * h: # if h_new shrink enough, then stop updating
            rho *= 10
        else:
            break
    mu += rho * h_new
    return rho, mu, h_new


def RKHS_nonlinear(model: nn.Module,
                    x: torch.Tensor,
                    lambda1: float = 0.,
                    mu: float = 0.,
                    max_iter: int = 100,
                    h_tol: float = 1e-8,
                    rho_max: float = 1e+16,
                    w_threshold: float = 0.1):
    rho, mu, h = 1.0, 0.0, np.inf
    for _ in range(max_iter):
        rho, mu, h = dual_ascent_step(model, x, lambda1, mu,
                                        rho, h, rho_max)
        if h <= h_tol or rho >= rho_max:
            break
    W_est = model.fc1_to_adj(x)
    W_est = W_est.detach().numpy()
    W_est[np.abs(W_est) < w_threshold] = 0
    W_est[np.abs(W_est) >= w_threshold] = 1
    return W_est

### Test

In [194]:
instance = NotearsRKHS(2, 3)
# Intialization
print(instance.fc1_pos.weight)
print(instance.fc1_pos.weight.bounds)

Parameter containing:
tensor([[0., 0.],
        [0., 0.],
        [0., 0.]], requires_grad=True)
[(0, None), (0, None), (0, None), (0, None), (0, None), (0, None)]


In [170]:
# forward
x = np.array([[1, 2, 3], [4, 5, 7]])
K1 = instance.gaussian_kernel_matrix_and_grad(x)[0]
print(f"K1: {K1}")

x = torch.tensor(x, dtype=torch.float32, requires_grad=True)
K2 = torch.zeros(2, 2)
for i in range(2):
      for l in range(2):
            K2[i,l] = gaussian_kernel(x[i,:], x[l,:])
print(f"K2: {K2}")
print(torch.equal(K1, K2))

x = np.array([[1, 2, 3], [4, 5, 7]])
print(instance.forward(x).shape)

K1: tensor([[1.0000e+00, 1.7139e-15],
        [1.7139e-15, 1.0000e+00]], grad_fn=<ExpBackward0>)
K2: tensor([[1.0000e+00, 1.7139e-15],
        [1.7139e-15, 1.0000e+00]], grad_fn=<CopySlices>)
False
torch.Size([2, 3])


In [171]:
instance.fc1_to_adj(x)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], grad_fn=<DivBackward0>)

In [146]:
x = np.array([[1, 2, 3], [4, 5, 7]])
instance.gaussian_kernel_matrix_and_grad(x)[1]

tensor([[[-0.0000e+00, -0.0000e+00, -0.0000e+00],
         [ 1.0283e-14,  1.0283e-14,  1.3711e-14]],

        [[-1.0283e-14, -1.0283e-14, -1.3711e-14],
         [-0.0000e+00, -0.0000e+00, -0.0000e+00]]], grad_fn=<MulBackward0>)

In [147]:
n = 2
d = 3
def grad_kernel(x) -> torch.Tensor: # [n, n, d] grad_storage[i, l, k]: gradient of k(x^i, x^l) wrt x^l_{k}
      #Get W from fc1 weights, take L2 norm of the gradient
      # get [n, n, d] gradient matrix of the kernel
      # Initialize a tensor to store gradients
      x = torch.tensor(x, dtype=torch.float32, requires_grad=True)
      grad_K = torch.zeros(n, n, d)
      # Compute gradients
      for i in range(n):
         for l in range(n):
            x_i = x[i,:]
            x_l = x[l,:]
            x_i.retain_grad()
            K_il = gaussian_kernel(x_i, x_l, gamma=1)
            # Zero out gradients in x, otherwise it accumulate the gradients
            if x_i.grad is not None:
                  x_i.grad.zero_()
            # Select individual scalar element from y
            # Perform backward pass on the scalar element
            K_il.backward()
            # Store the computed gradients
            grad_K[i, l] = x_i.grad.clone() 
      return grad_K

grad_kernel(x)


tensor([[[-0.0000e+00, -0.0000e+00, -0.0000e+00],
         [ 1.0283e-14,  1.0283e-14,  1.3711e-14]],

        [[-1.0283e-14, -1.0283e-14, -1.3711e-14],
         [-0.0000e+00, -0.0000e+00, -0.0000e+00]]])

In [148]:
# L_risk 
x = np.array([[1, 2, 3], [4, 5, 7]])
fc1_weight = torch.tensor([[3,7,4], [6, 1, 5]], dtype=torch.float32, requires_grad=True).t()
penalty = 0.1
K = instance.gaussian_kernel_matrix_and_grad(x)[0] # [n, n]
x = torch.tensor(x, dtype=torch.float32, requires_grad=True)
temp = torch.matmul(torch.matmul(fc1_weight, K), fc1_weight.t())
diagonal = torch.sum(torch.diag(temp))
regularized1 = penalty*diagonal 
print(regularized1)

regularized2 = 0
for j in range(fc1_weight.shape[0]):
    for m in range(fc1_weight.shape[1]):
        for l in range(fc1_weight.shape[1]):
            regularized2 += fc1_weight[j, m]*fc1_weight[j, l]*K[m,l]
regularized2 = penalty*regularized2
print(regularized2)
print(torch.equal(regularized1, regularized2))


tensor(13.6000, grad_fn=<MulBackward0>)
tensor(13.6000, grad_fn=<MulBackward0>)
True


In [149]:
# weight
x = np.array([[1, 2, 3], [4, 5, 7]])
x = torch.tensor(x, dtype=torch.float32, requires_grad=True)
grad_K = torch.tensor([[[3,7,4], [6, 1, 5]], 
                       [[1, 2, 3], [4, 5, 7]]], dtype=torch.float32, requires_grad=True)

def fc1_to_adj(x, grad_K) -> torch.Tensor: # [d, d]
       grad_K = grad_K.transpose(0, 2).unsqueeze(-1) # [d ,n, 1]
       weight = torch.matmul(fc1_weight, grad_K).squeeze(-1) # [d, n, d]
       #weight = torch.sum(weight ** 2, dim = 1)/n # [d, d]
       return weight

fc1_to_adj(x, grad_K)
#print(fc1_to_adj(x, grad_K).shape)

tensor([[[15., 22., 17.],
         [42., 46., 44.]],

        [[33., 51., 38.],
         [33., 12., 29.]],

        [[30., 31., 31.],
         [57., 42., 55.]]], grad_fn=<SqueezeBackward1>)

In [150]:
d = 3
n = 2
storage = torch.zeros(d, n, d)
for k in range(0, d):
    for l in range(0,n):
        for j in range(0, d):
            for i in range(0, n):
                storage[k,l,j] += grad_K[i,l,k]*fc1_weight[j, i]
print(storage)


tensor([[[15., 22., 17.],
         [42., 46., 44.]],

        [[33., 51., 38.],
         [33., 12., 29.]],

        [[30., 31., 31.],
         [57., 42., 55.]]], grad_fn=<CopySlices>)


In [175]:
RKHS_nonlinear(instance, x)

1.0
1e+16


array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)

### Learning causality by self generated data

#### linear case

In [198]:
np.random.seed(2)
x = np.random.normal(0,1, 400)

np.random.seed(24)
l = np.random.normal(0,1, 400) 
y = [0.5*a + b for a, b in zip(x, l)]

matrix = np.column_stack((x, y))
n = matrix.shape[0]
d = matrix.shape[1]

print(n)
print(d)

400
2


In [199]:
example = NotearsRKHS(n, d)


In [200]:
RKHS_nonlinear(example, matrix)

1.0
1e+16
1.0
1e+16
100.0
1e+16
1000.0
1e+16
10000.0
1e+16
1000000.0
1e+16


array([[0., 1.],
       [0., 0.]], dtype=float32)

#### non-linear case

In [201]:
np.random.seed(2)
x = np.random.normal(0,1, 400)

np.random.seed(24)
l = np.random.normal(0,1, 400) 
 
y = [a**2 + b for a, b in zip(x, l)]

matrix = np.column_stack((x, y))
n = matrix.shape[0]
d = matrix.shape[1]

In [202]:
non_linear_example = NotearsRKHS(n, d)
RKHS_nonlinear(non_linear_example, matrix)

1.0
1e+16
1.0
1e+16
100.0
1e+16
1000.0
1e+16
100000.0
1e+16


array([[0., 1.],
       [0., 0.]], dtype=float32)

In [203]:
np.random.seed(2)
x = np.random.normal(0,1, 400)

np.random.seed(24)
l = np.random.normal(0,1, 400) 
 
y = [np.sin(a) + b for a, b in zip(x, l)]

matrix = np.column_stack((x, y))
n = matrix.shape[0]
d = matrix.shape[1]

In [204]:
non_linear_example = NotearsRKHS(n, d)
RKHS_nonlinear(non_linear_example, matrix)

1.0
1e+16
1.0
1e+16
100.0
1e+16
1000.0
1e+16
100000.0
1e+16
10000000.0
1e+16


array([[0., 1.],
       [0., 0.]], dtype=float32)